In [8]:
import os,glob
import numpy as np
import SimpleITK as sitk
from math import fabs, copysign
from collections import OrderedDict
import shutil
from utils import *
import operator

ModuleNotFoundError: No module named 'batchgenerators'

In [2]:
data_file = 'F:/data/petmrBrain'

In [3]:
data_list = sorted(os.listdir(os.path.join(data_file,'raw_mask')))
print (len(data_list))
print (data_list[0])
name_list = [name.split('_')[-1] for name in data_list]
print (len(name_list))
print (name_list[0])

120
mask_002.nii.gz
120
002.nii.gz


In [6]:
def generate_datainfo_csv(name,ppty):
 
    #name_attribute = ['id','class_num','size','ori_spacing','origin','spacing','direction','class']

    img = dict()
    img['id'] = name
    
    img['class_num'] = ppty['class_num']
    img["size"] = ppty['original_size_of_raw_data']
    img["ori_spacing"] = ppty['original_spacing']
    img['origin'] = ppty['itk_origin']
    img['spacing'] = ppty['itk_spacing']
    img['direction'] = ppty['itk_direction']
    img['class'] = ppty['class']
    
    return img

In [ ]:
def generate_datainfo_pkl(name,data_file,pet,mri,mask,class_num):

    property = OrderedDict()
    property['id'] = name
    
    property['ori_pet'] = os.path.join(data_file,'raw_pet','pet_'+ name+'.nii.gz')
    property['ori_mri'] = os.path.join(data_file,'raw_mri','mri_'+ name+'.nii.gz')
    property['ori_mask'] = os.path.join(data_file,'raw_mask','mask_'+ name+'.nii.gz')
    
    property['class_num'] = len(class_num)
    property['class'] = class_num
    
    property["original_size_of_raw_data"] = np.array(pet.GetSize())[[2, 1, 0]]
    property["original_spacing"] = np.array(pet.GetSpacing())[[2, 1, 0]]
    property["itk_origin"] = pet.GetOrigin()
    property["itk_spacing"] = pet.GetSpacing()
    property["itk_direction"] = pet.GetDirection()
    
    return property

In [7]:
csv_info_list = []
all_properties = OrderedDict()

for file_name in name_list:
    name = file_name.split('.')[0]
    #print ('name:',name)
    pet_itk = sitk.ReadImage(os.path.join(data_file,'raw_pet','pet_'+file_name))
    mri_itk = sitk.ReadImage(os.path.join(data_file,'raw_mri','mri_'+file_name))
    mask_itk = sitk.ReadImage(os.path.join(data_file,'raw_mask','mask_'+file_name))
    
    mask_np = sitk.GetArrayFromImage(mask_itk).astype(np.int32)
    class_num = np.unique(mask_np)
    
    # pkl
    properti = generate_datainfo_pkl(name,data_file,pet_itk,mri_itk,mask_itk,class_num)
    all_properties[name] = properti
    
    #csv
    csv_info_list.append(generate_datainfo_csv(name,properti))
    

--------------002--------------
pet info: [array([-123.610054,  123.54262 ,  127.677216], dtype=float32), array([1, 1, 1]), array([ 1,  0,  0,  0,  0, -1,  0, -1,  0])]
mri_itk_info: [array([-123.610054,  123.54262 ,  127.677216], dtype=float32), array([1, 1, 1]), array([ 1,  0,  0,  0,  0, -1,  0, -1,  0])]
mask_itk_info: [array([-123.610054,  123.54262 ,  127.677216], dtype=float32), array([0, 0, 0]), array([ 1,  0,  0,  0,  0, -1,  0, -1,  0])]


AssertionError: 

In [ ]:
name_attribute = ['id','class_num','size','ori_spacing','origin','spacing','direction','class']
save_csv(csv_info_list,'pet_mri_Brain_info.csv',name_attribute)

In [ ]:
save_pickle(all_properties,'pet_mri_Brain_info.pkl')